In [1]:
import pandas as pd
import numpy as np
import requests
import urllib
from lxml.html import fromstring
import threading
import concurrent.futures
from collections import defaultdict
import re
import logging

In [2]:
# Read in seed spreadsheet
jelitto = pd.read_excel('jelitto_pricelist.xls')

In [3]:
# Don't truncate text
pd.set_option('display.max_colwidth', -1)

In [4]:
# Subset of rows to test
jelitto = jelitto.iloc[0:250, :]

In [5]:
# Check dataframe
jelitto.head()

,Characteristic (GOLD NUGGET SEED® etc.),Herbs/Ornamental Grasses,Item No.,Genus,Species,Series,Variety,Marketing Name,Synonyme,Common Names,...,Height to (cm),Flowering from,Flowering to,Hardiness Zone from,Hardiness Zone to,g/1.000 Plants,Flower Habit,Catalogue Description,Restrictions,Alphabetical Sorting
0,NaN,NaN,AA008,ABUTILON,vitifolium,NaN,NaN,NaN,NaN,"Indian Mallow, Flowering Maple",...,150,May,June,Z8,Z11,10.0,NaN,"mostly blue, rarely white, large Mallow flowers",NaN,10002000
1,NaN,NaN,AA001,ACAENA,buchananii,NaN,NaN,NaN,NaN,"New Zealand Bur, Bidibidi, Piripiri",...,10,July,August,Z5,Z8,5.0,NaN,blue-green foliage,Australia:prohibited,10003500
2,NaN,NaN,AA002,ACAENA,caesiiglauca,NaN,NaN,NaN,NaN,"New Zealand Bur, Bidibidi, Piripiri",...,10,July,August,Z5,Z8,5.0,NaN,grey-green leaves,Australia:prohibited,10004000
3,NaN,NaN,AA011,ACAENA,fissistipula,NaN,NaN,NaN,NaN,New Zealand Burr,...,10,July,August,Z5,Z8,5.0,NaN,"fine, blue-green leaves, white flowers with red anthers",Australia:prohibited,10004500
4,NaN,NaN,AA003,ACAENA,inermis,NaN,NaN,NaN,NaN,"New Zealand Burr, Sheep's Burr",...,10,July,August,Z5,Z8,8.0,NaN,"large, unprickled blossom heads, green foliage",Australia:prohibited,10005000


In [6]:
# Add row for image url
image_url = jelitto['Item No.'].apply(lambda row: 'https://www.jelitto.com/out/pictures/master/product/1/' + row.lower() + '.jpg')


In [7]:
def response_code_error(url):
    """Confirm that Jelitto has an image of the plant."""
    try:
        return urllib.request.urlopen(url).getcode()
    except urllib.error.URLError:
        return np.nan

In [8]:
# Save checked urls to a dictionary.
confident_urls = defaultdict(str)

# Threaded Jelitto image url checking.
with concurrent.futures.ThreadPoolExecutor(max_workers=25) as executor:
    check_url = {executor.submit(response_code_error, url): url for url in image_url}
    for future in concurrent.futures.as_completed(check_url):
        url = check_url[future]
        try:
            data = future.result()
            if data == 200:
                confident_urls[re.search(r'\/1\/(.*)\.', url).group(1)] = url
            else:
                confident_urls[re.search(r'\/1\/(.*)\.', url).group(1)] = np.nan 
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))


In [9]:
# Cast confident_urls to a dataframe sorted by index (the Item No.)
df = pd.DataFrame.from_dict(confident_urls, orient='index', columns=['url'])
df.sort_index(inplace=True)

In [10]:
# Set the index of the jelitto df to Item No & sort
jelitto.set_index('Item No.', inplace=True)
jelitto.sort_index(inplace=True)


In [11]:
# Add image_url column to jelitto
jelitto['image_url'] = df['url'].values

In [12]:
# Create list of wikimedia_urls for rows where jelitto had no image 
# and create alternate lookups for the common name to check if the scientific name has no results
wikimedia_urls = []
alternate_lookups = {}
_url = 'https://commons.wikimedia.org/w/index.php?search='
updates = jelitto[jelitto['image_url'].isnull()]
wikimedia_urls = [(ix, _url + x) for ix, x in updates[['Genus', 'Species ']].apply(lambda x: ' '.join(x).lower(), axis=1).iteritems()]
alternate_lookups = {i: _url+'+'.join(x.split(',')[0].split()) for i,x in updates['Common Names'].iteritems() if not isinstance(x, float)}
alternate_urls = []

In [13]:
def alt_url(url, scientific=False):
    """Test tuple of _id and url to find an image on wikimedia commons."""
    _id = url[0]
    url = url[1]
    page = requests.get(url)
    text = fromstring(page.content)
    try:
        new_url = text.xpath("//li[@class='mw-search-result']//a/@href|//ul[contains(@class, 'gallery')]//img/@src")[0]
        if new_url.lower().endswith(('.png', '.jpg', '.jpeg')):
            if (_id, "https://commons.wikimedia.org"+new_url) not in alternate_urls:
                alternate_urls.append((_id, "https://commons.wikimedia.org"+new_url))
            if scientific:
                del alternate_lookups[_id]
        else:
            if scientific:
                pass # try common name
            else:
                alternate_urls.append((_id, np.nan)) 
    except Exception as e:
        logging.info(f"Exception {e} encountered at {_id} {url}") 
        if scientific:
            pass
        else:
            alternate_urls.append((_id, np.nan)) 

In [14]:
# Checking wikimedia urls with scientific name lookup
with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    # Start the load operations and mark each future with its URL
    check_url = {executor.submit(alt_url, url, scientific=True): url for url in wikimedia_urls}  
    for future in concurrent.futures.as_completed(check_url):
        url = check_url[future]

In [15]:
# Checking wikimedia common name lookup
if alternate_lookups:
    lookups = [(k,v) for k,v in alternate_lookups.items()]
    with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
        check_url = {executor.submit(alt_url, url): url for url in lookups}
        for future in concurrent.futures.as_completed(check_url):
            url = check_url[future]

In [16]:
# Cast alternate_urls to df with index aligned to jelitto
altdf = pd.DataFrame(alternate_urls)
altdf.rename({0: '_id', 1: 'image_url'}, axis=1, inplace=True)
altdf.set_index('_id', inplace=True)


In [17]:
# Replace null values in image_url with urls from alternate_urls
jelitto = jelitto.combine_first(altdf)

In [18]:
# Check results
jelitto[['Genus', 'Species ', 'image_url']]

,Genus,Species,image_url
AA001,ACAENA,buchananii,https://www.jelitto.com/out/pictures/master/product/1/aa001.jpg
AA002,ACAENA,caesiiglauca,https://www.jelitto.com/out/pictures/master/product/1/aa002.jpg
AA003,ACAENA,inermis,https://www.jelitto.com/out/pictures/master/product/1/aa003.jpg
AA005,ACAENA,microphylla,https://www.jelitto.com/out/pictures/master/product/1/aa005.jpg
AA007,ACAENA,novae-zelandiae,https://www.jelitto.com/out/pictures/master/product/1/aa007.jpg
AA008,ABUTILON,vitifolium,https://www.jelitto.com/out/pictures/master/product/1/aa008.jpg
AA009,ACAENA,saccaticupula,https://www.jelitto.com/out/pictures/master/product/1/aa009.jpg
AA011,ACAENA,fissistipula,https://www.jelitto.com/out/pictures/master/product/1/aa011.jpg
AA012,ACANTHUS,hungaricus,https://www.jelitto.com/out/pictures/master/product/1/aa012.jpg
AA014,ACANTHUS,mollis,https://www.jelitto.com/out/pictures/master/product/1/aa014.jpg


In [ ]:
# Output to csv
jelitto.to_csv('')